# Bootstrapping

1. What is bootstraping?
2. An example in Python
3. Why bootstrap?
4. Confidence Intervals
5. References

## What is bootstrapping?

Bootstrapping is repeated resampling with replacement, often for the purpose of estimating the standard error of a statistic

** Repeated resampling with replacement **

Imagine that you have a fruit basket with four fruit in it (your sample). You take out a fruit, record the value of that fruit, and put it back in the basket. Then you take out another fruit, record what type it is, and put it back in the basket.

In [ ]:
import numpy as np
np.random.seed(100)
sample = np.array(['apple', 'orange', 'pear', 'bananna'])
resample1 = np.random.choice(sample, size = 4, replace = True)
resample2 = np.random.choice(sample, size = 4, replace = True)
resample3 = np.random.choice(sample, size = 4, replace = True)

In [ ]:
sample

In [ ]:
resample1

In [ ]:
resample2

In [ ]:
resample3

** Statistic **

What is the difference between a statistic and a parameter? A statistic is a property of the sample, while a parameter is a property of the distribution. Here a few examples that you will recognize:

- $\hat{\theta}=min(X)$
- $\hat{\theta}=\bar{X}$
- $\hat{\theta}=median(X)$
- $\hat{\theta}=\hat{\beta_1}$ (Linear regression coefficient)
- $\hat{\theta}=R^2$ (Coefficient of determination)

In [ ]:
sample = np.array([1, 2, 3, 4, 5, 6, 12])
print('Min:', np.min(sample))
print('Max:', np.max(sample))
print('Mean:', np.mean(sample))
print('Variance:', np.var(sample))
print('Median:', np.median(sample))

Here is a diagram that shows how these two concepts work togther.
![title](bootstrapping.png)

**Estimating the Standard Error**
> Standard error is a general term for the standard deviation of a summary statistic. They are the most common way of determining statistical accuracy.

Efron and Tibshirani, An Introduction to the Bootstrap

We need to estimate the standard error of a statistic, because we usually won't know the parameters of the distribution that the statistic is summarized from.

** Putting it all together **

Here is a simple three-step algorithm:
1. Draw B bootstrapped samples from the original sample
2. Calculate the statistic for each of the bootstrapped samples
3. Estimate the standard error by calculating the standard deviation of the statistics

This algorithm is adapted from p. 47 of An Introduction to the Bootstrap

## An example in Python

Let's drive this home with a simple example in python. In this example we are going to estimate the standard error of the mean statistic. 

In [ ]:
np.random.seed(100)
sample = np.random.normal(loc = 10, scale = 3, size = 15)
sample

In [ ]:
# Estimate of the statistic
thetaHat = np.mean(sample)
thetaHat

In [ ]:
n = sample.shape[0]
B = 1000
statBoot = np.zeros(B)

for b in range(B):
    resample = np.random.choice(sample, size = n, replace = True)
    statBoot[b] = np.mean(resample)

Let's inspect the result through visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
def plot_hist(array, ax):
    '''Plot a histogram of bootstrapped values'''
    kwargs = dict(bins=30, normed=True, color='steelblue', edgecolor='none')
    ax.hist(array, **kwargs)
    ax.set(title='Bootstrapping', ylabel='Density', xlabel = 'Statistic')

In [ ]:
fig, ax = plt.subplots(1)
plot_hist(statBoot, ax)
ax.axvline(x = thetaHat, linestyle = '--', color = 'black')

In [ ]:
# Estimate of the standard error
np.sqrt(np.var(statBoot))

## Why bootstrap?
There is a closed form equation for estimating the standard error of the sample mean  is easily derived in mathmatical closed form for the mean: $$\hat{se}(\bar{X})=\frac{s}{\sqrt(n)}$$

In [ ]:
# Closed-form estimate of the standard error
np.sqrt(np.var(sample) / n) 

So why bother with the bootstrap? 

The answer is that the standard error for many statistics is not so easy to derive mathmatically. For example, it is very difficult to mathmatically derive the standard error for the sample median.

## Calculating Confidence Intervals
One of the main applications of the bootstrap is to calculate confidence intervals, which can be used to quantify a range of possible values for your parameter. 

As a reminder, the way to interprete a 95% confidence interval is that 95% of the time, the confidence interval will trap the true parameter.

![title](confidence_intervals.png)

** Calculating Normal Intervals **

The easiest way to calculate a confidence interval is to assume that your sampling statistic follows a normal distribution: 
$$\hat{\theta} \pm 1.96\hat{se}$$

Where z = 1.96 is the z value associated with a 95% confidence interval and $\hat{se}$ is the estimated standard error from the bootstrapping. 

In our toy example, this is how we would compute the confidence interval:

In [ ]:
thetaHat = np.mean(sample)
seHat = np.sqrt(np.var(statBoot))
[thetaHat - 1.96 * seHat, thetaHat + 1.96 * seHat]

However, this method does not always work. Let's examine a more complicated example where this normal method breaks down.

In this example the statistic we are estimating is the correlation between average LSAT scores and the GPA for law schools. In this example, the population is 82 law schools, and our sample is 15 of those law schools.

In [ ]:
import pandas as pd
law = pd.read_csv('law_sample.csv')
law

In [ ]:
sample = law.values
n = sample.shape[0]
B = 1000
statBoot = np.zeros(B)

In [ ]:
def calc_corr(array):
    '''Calculate pearsons correlation coefficient for 1st and 
    2nd column in an array'''
    return(np.corrcoef(array[:,1], array[:,2])[0, 1])

In [ ]:
np.random.seed(100)
for b in range(B):
    idx = np.random.choice(range(n), size = n, replace = True)
    resample = sample[idx,:]
    statBoot[b] = calc_corr(resample)

In [ ]:
statBoot[:10]

Now let's calculate the confidence interval as before, using the normal method

In [ ]:
thetaHat = calc_corr(sample)
seHat = np.sqrt(np.var(statBoot))
print(thetaHat)
normalCI = [thetaHat - 1.96 * seHat, thetaHat + 1.96 * seHat]
normalCI

Here we come to our first of two issues. The correlation coefficient is only defined on the range [-1, 1], yet the normal confidence interval does not preserve this range property.

To see the second issue, let's take a look at the distribution of the statistic.

In [ ]:
fig, ax = plt.subplots(1)
plot_hist(statBoot, ax)

As you can see, the distribution is not normally shaped. So it doesn't really make sense to use the normal confidence intervals here for that reason either.

**Percentile Confidence Intervals**

A more robust method for calculating confidence intervals using the bootstrap is to use the percentiles of the bootstrapped distribution:
- Percentile method respects the range of the statistic
- Does not assume that the distribution of the statistic is normally distributed

We can calculate the percentiles 2.5%, for example by identifying the value that 2.5% of the sampled statistics are less than.

In [ ]:
percentileCI = np.percentile(statBoot, [2.5, 97.5])
percentileCI

In [ ]:
fig, ax = plt.subplots(1)
plot_hist(statBoot, ax)
ax.axvline(x = thetaHat, color = 'black')
ax.axvline(x = percentileCI[0], linestyle = '--', color = 'black')
ax.axvline(x = percentileCI[1], linestyle = '--', color = 'black')

## References
1. **Efron, B. and Tibshirani, R. (1993) An Introduction to the Bootstrap. Chapman and Hall, New York, London.** The definitive guide to Bootstrapping written by the inventor of the Bootstrap and one of his most prominent graduate students ([Amazon link](https://www.amazon.com/Introduction-Bootstrap-Monographs-Statistics-Probability/dp/0412042312/ref=sr_1_1?ie=UTF8&qid=1484328384&sr=8-1&keywords=an+introduction+to+the+bootstrap)).

2. **Freedman, D., Pisani, R. and Purves, R. (2007) Statistics. W. W. Norton & Company, New
York, London.** Widely considered to be "The Bible" for teaching college statistics, this is a great book for anyone who didn't get the chance to take Statistics as an undergraduate level, or who simply wants a refresher on basic concepts ([Amazon link](https://www.amazon.com/Statistics-4th-David-Freedman/dp/0393929728/ref=sr_1_3?ie=UTF8&qid=1484328391&sr=8-3&keywords=statistics)).